In [1]:
import sys

from morpheus.project.types.geometry.MultiPolygon import MultiPolygon

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid, ActiveCells
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()

grid = Grid.cartesian_from_polygon(
  polygon=polygon,
  n_rows=2000,
  n_cols=2000,
  rotation=rotation,
)
polygon

Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96609040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon')


## Performance of ActiveCells Calculation with a Point

In [3]:
%%time
from morpheus.project.types.geometry import Point

point_within = Point(
  type='Point',
  coordinates=(13.923, 50.965)
)

ac = ActiveCells.from_point(point=point_within, grid=grid)
len(ac)




CPU times: user 130 ms, sys: 356 ms, total: 486 ms
Wall time: 68.3 ms


1

## Performance of ActiveCells Calculation with a LineString 

In [4]:
from morpheus.project.types.geometry import LineString

linestring_within = LineString(
  type='LineString',
  coordinates=[
    (13.923, 50.965),
    (13.924, 50.965),
  ]
)

In [5]:
%%time
from morpheus.project.types.discretization.spatial import ActiveCells

ac = ActiveCells.from_linestring(linestring=linestring_within, grid=grid)
len(ac)

CPU times: user 402 ms, sys: 4.17 ms, total: 406 ms
Wall time: 341 ms


590

## Performance of ActiveCells Calculation with a Polygon

In [6]:
from morpheus.project.types.geometry import Polygon

polygon_within = Polygon(
  type='Polygon',
  coordinates=[[
    (13.923, 50.965),
    (13.924, 50.9655),
    (13.925, 50.965),
    (13.925, 50.966),

    (13.923, 50.966),
    (13.923, 50.965),
  ]]
)

In [7]:
import folium

m = folium.Map(location=[50.9655, 13.9235], zoom_start=15)
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(polygon_within.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m


In [8]:
%%time
active_cells = ActiveCells.from_polygon(polygon=polygon_within, grid=grid)
len(active_cells)

CPU times: user 836 ms, sys: 17.2 ms, total: 853 ms
Wall time: 1.15 s


1270195

## Performance of ActiveCells Calculation with a MultiPolygon

In [9]:
%%time
multi_polygon_within = MultiPolygon(
  type='MultiPolygon',
  coordinates=[
    [[
      (13.923, 50.965),
      (13.924, 50.9655),
      (13.925, 50.965),
      (13.925, 50.966),
      (13.923, 50.966),
      (13.923, 50.965),
    ]],
    [[
      (13.9235, 50.9655),
      (13.924, 50.966),
      (13.9245, 50.9655),
      (13.9235, 50.9655),
    ]]
  ]
)

active_cells = ActiveCells.from_multipolygon(multipolygon=multi_polygon_within, grid=grid)
len(active_cells)

CPU times: user 1.24 s, sys: 22.5 ms, total: 1.26 s
Wall time: 1.39 s


1270195

## Time to generate the outline of the active cells

In [10]:
active_cells = ActiveCells.from_polygon(polygon=polygon_within, grid=grid)
len(active_cells)

1270195

In [11]:
%%time
active_cells_outline = active_cells.outline_to_geojson(grid=grid)
print(active_cells_outline)

Feature(geometry=Polygon(coordinates=(((13.922998303121226, 50.96566509282753), (13.922998325350129, 50.965665099088035), (13.922998481874709, 50.965665134967), (13.922998576546583, 50.96566516163021), (13.92299866814126, 50.96566519746464), (13.922998755772676, 50.965665242123634), (13.922998838593116, 50.965665295175164), (13.922998915801402, 50.965665356106044), (13.922998986650642, 50.96566542432683), (13.922999050455465, 50.96566549917759), (13.922999106598638, 50.965665579934225), (13.922999154537054, 50.965665665815536), (13.92299919380697, 50.96566575599072), (13.922999224028501, 50.96566584958747), (13.922999244909294, 50.96566594570033), (13.922999256247353, 50.96566604339956), (13.922999257932998, 50.96566614174004), (13.922999249949923, 50.96566623977045), (13.922999232375354, 50.965666336542476), (13.9229992053793, 50.96566643111998), (13.922999169222914, 50.965666522588045), (13.922999124255961, 50.96566661006183), (13.922999008145258, 50.965666810970816), (13.92299862358

In [12]:
import folium

m = folium.Map(location=[50.9655, 13.9235], zoom_start=15)
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(polygon_within.as_geojson()).add_to(m)
folium.GeoJson(active_cells_outline.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m